In [37]:
import numpy as np
import pandas as pd


In [38]:
#Function Definition/ class designation

#Equation for x_d
def x_d_equation(x_hat,gamma):
    return x_hat-np.tan(gamma*np.pi/180)
#Full equation estimator given known information
def epsilon_s(es_0,es_beta,beta):
    return es_0 + es_beta*beta*(np.pi/180)

def epsilon_beta(Cl_w,Ra_w,b_w,K_v,K_b,x,y,z,gamma):
    #solve for x_hat,y_hat,z_hat,x_d
    x_hat = (2*x)/(b_w*K_b)
    y_hat = (2*y)/(b_w*K_b)
    z_hat = (2*z)/(b_w*K_b)
    x_d = x_d_equation(x_hat,gamma)
    #lead coefficient
    lead_co = -(K_v*x_d*Cl_w)/(Ra_w*K_b*np.pi**2)
    var1 = ((-2*y_hat*(z_hat-1))/(y_hat**2 + (z_hat-1)**2)**2)*(1+((x_d)/(np.sqrt(x_d**2 + y_hat**2 + (z_hat-1)**2))))
    var2 = ((2*y_hat*(z_hat+1))/(y_hat**2 + (z_hat+1)**2)**2)*(1+((x_d)/(np.sqrt(x_d**2 + y_hat**2 + (z_hat+1)**2))))
    var3 = (y_hat/(y_hat**2 + (z_hat-1)**2))* ((x_d*(z_hat-1))/((x_d**2 + y_hat**2 + (z_hat-1)**2)**(3/2)))
    var4 = (y_hat/(y_hat**2 + (z_hat+1)**2))* ((x_d*(z_hat+1))/((x_d**2 + y_hat**2 + (z_hat+1)**2)**(3/2)))
    return lead_co*(var1+var2-var3+var4)

def epsilon_beta_simplified(CL_w,RA_w,K_v,K_beta,K_b):
    return -(CL_w*K_v*K_beta)/(K_b*RA_w)

#function to get the K_beta value using the equation where it lies on axis of sym ie Z=0
def get_K_beta_val_Z_zero(K_b,b_w,gamma,x,y):
    x_hat = (2*x)/(b_w*K_b)
    y_hat = (2*y)/(b_w*K_b)
    x_d = x_d_equation(x_hat,gamma)
    var1 = ((4*y_hat*x_d)/(np.pi**2 *(y_hat**2 + 1)**2)) *(1+((x_d)/(np.sqrt(x_d**2 + y_hat**2 +1 ))))
    var2 = ((2*y_hat)/(np.pi**2 *(y_hat**2 + 1)**2)) *((x_d**2)/((x_d**2 + y_hat**2 +1)**(3/2)))
    return var1 + var2

def get_CL_v_effective(RA_eff,K_L,C_L_alpha_tilda):
    return C_L_alpha_tilda/((1+C_L_alpha_tilda/(1*np.pi*RA_eff))*(1+K_L))

def get_CL_val(W,gamma,rho,vinf,sw):
    return W*np.cos(gamma*np.pi/180)/(0.5*rho*vinf**2 *sw)

def get_K_s_val(X,Y,B_w,K_b,Lambda_deg):
    Lambda_rad = Lambda_deg*np.pi/180
    X_bar = X/(B_w*0.5)
    Y_bar = Y/(B_w*0.5)
    r_bar = np.sqrt(X_bar**2+Y_bar**2)
    s_bar = K_b*np.tan(Lambda_rad)
    t_bar = np.sqrt((X_bar-s_bar)**2+Y_bar**2+K_b**2)
    t_0bar = np.sqrt(X_bar**2+Y_bar**2+K_b**2)
    top = 1 + (X_bar-s_bar)/(t_bar) + (X_bar*(r_bar+t_bar)*(t_0bar**2 - X_bar**2))/(r_bar*t_bar*(r_bar*t_bar+r_bar**2 -X_bar*s_bar))
    bottom = 1 + (X_bar*(r_bar**2 + t_0bar**2 -X_bar**2))/(t_0bar*r_bar**2 )
    K_s = top/bottom
    return K_s


In [48]:
class Wing_part:
    
    def __init__(self,C_barW,B_W,weight,launch_force = 20):
        self.span = B_W
        self.mean_chord = C_barW
        self.aspect = self.span/self.mean_chord
        self.plan_area = self.span*self.mean_chord
        self.launch_force = launch_force #must be in ft-lbf (kinetic energy)
        self.gravity = 32.174 # ft/s^2 @ 5000 ft elevation
        self.weight = weight #lbf 
        self.air_rho = 0.002048 #slug/ft^3 for air at 5000 ft 
        self.dynam_vis = 3.6366 *10**(-7)
    def launch_velocity(self):
        velocity = np.sqrt(2*self.launch_force*self.gravity/self.weight)
        self.velocity = velocity
        return self.velocity
    def get_CL_val(self,gamma,velocity):
        self.CL =  self.weight*np.cos(gamma*np.pi/180)/(0.5*self.air_rho*velocity**2 *self.plan_area)
        return self.CL
    def get_RA_allowed(self,sigma_max,W_fuse,W_wing,gamma,t_max_c,b):
        self.Max_RA = ((sigma_max*W_wing*16*t_max_c)/(21*W_fuse*b*gamma))
        return self.Max_RA
    def best_no_wind_glide_vel(self,CD_0,e):
        val = (2**(0.5)/(np.pi*e*self.aspect*CD_0)**(0.25))*np.sqrt(self.weight/(self.plan_area*self.air_rho))
        return val
    
MainWing = Wing_part(0.75,9,1.10017)

print(MainWing.launch_velocity())
print(MainWing.get_CL_val(0,MainWing.velocity))
print(MainWing.get_RA_allowed(7200,0.6545,0.33461,0.804,0.12,9))
print(MainWing.best_no_wind_glide_vel(0.013,0.8))
MainWing.velocity = MainWing.best_no_wind_glide_vel(0.013,0.8)
print(MainWing.get_CL_val(0,MainWing.velocity))

34.20208841198289
0.1360663760705857
46.50987124836365
15.943619449769836
0.6261555423119768
